# Sieci splotowe (Convolutional Neural Networks)

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

### Przygotowanie danych
Wykorzystamy w zadaniu zbiór Fashion-MNIST. Jak zwykle zaczynamy od pobrania danych, sprawdzamy krótko jak wyglądają, a następnie definiujemy miarę trafności

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST

target_directory = "fmnist"

fmnist_real_train = FashionMNIST(target_directory, train=True, download=True, transform=transforms.ToTensor())
fmnist_test = FashionMNIST(target_directory, train=False, download=True, transform=transforms.ToTensor())

In [ ]:
fmnist_train, fmnist_validation = data.random_split(fmnist_real_train, (48000, 12000))
len(fmnist_train), len(fmnist_validation)

In [ ]:
fmnist_train[0][0].shape

In [ ]:
trainloader = data.DataLoader(fmnist_train, batch_size=64, shuffle=True)
dataiter = iter(trainloader)
images,_ = dataiter.next()
plt.figure(figsize=(5,5))
for k in range(12):
    plt.subplot(3, 4, k+1)
    plt.imshow(images[k].numpy().squeeze(), cmap='gray_r')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
def compute_acc(logits, expected):
    pred = logits.argmax(dim=1)
    return (pred == expected).type(torch.float).mean()

#### Warstwa splotowa (konwolucyjna)

Warstwa splotowa przemieszcza jądro (ang. *kernel*) po obrazie kawałek po kawałku, oblicza wynik i zapamiętuje go w macierzy wyjściowej. Przetwarzany obrazek jest często uzupełniany o zera (ang. *padding*), ponieważ bez tego macierz wynikowa byłaby mniejsza niż obrazek wejściowy. Piksel w macierzy wyjściowej obliczany jest następująco: każdy z pikseli obrazu wejściowego jest mnożony przez odpowiadającą mu wartość w filtrze, a tak uzyskane 9 wartości jest sumowane. Następnie ten sam filtr wykorzystywany jest do obliczenia kolejnego piksela. Filtr składa się z wag splotowej sieci neuronowej, co oznacza, że to on podlega uczeniu.

Jądro nie musi przesuwać się za każdym razem o 1 piksel, może mieć większy krok (ang. *stride*).

Jeśli obrazek jest kolorowy możemy go postrzegać jako kilka obrazków monochromatycznych.

Przykład: [jak działa sieć konwolucyjna](https://bfirst.tech/wp-content/uploads/2019/06/sieci_konwolucyjne_gif-1.gif)

Warstwę splotową w PyTorch realizuje klasa [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/nn.html?highlight=conv2d#torch.nn.Conv2d). Pierwsze jej trzy parametry są obowiązkowe, są to:

- liczba map na wejściu, 
- liczba map na wyjściu, 
- rozmiar jądra (jedna liczba jeżeli ma być kwadratowe albo para liczb jeżeli ma być prostokątem).

Będziemy budowali krok po kroku listę, w której będziemy umieszczali kolejne warstwy sieci.

Rozpoczniemy od dodania warstwy splotowej.

Obrazki MNIST są monochromatyczne, więc mamy tylko 1 kanał wejściowy.

Przyjmimy, że na wyjściu będziemy mieli **5 map**, każdą na bazie kwadratowego filtra o boku 3.
Żeby nie zmniejszyć zbyt szybko obrazka dodamy po 1 pikselu paddingu z każdej strony - jak w ww. przykładzie.

In [ ]:
layers = [nn.Conv2d(1, 5, 3, padding=1)]

Warstwa splotowa - podobnie jak warstwa liniowa (`nn.Linear`) - jest tylko sumą. By wprowadzić nieliniowość, zastosujemy *Leaky ReLU*

In [ ]:
layers.append(nn.LeakyReLU())

Druga warstwa jest nazywana łączącą (ang. *pooling layer*). Jej zadaniem jest zmniejszenie wymiarów cech konwolucyjnych, wyznaczonych w poprzedniej warstwie, przy zachowaniu kluczowych szczegółów. Odpowiada również za redukcję szumu. 

Warstwa ta, podobnie jak warstwa splotowa przesuwa filtr przez obraz, ale ten filtr jest pozbawiony parametrów: służy albo do wybierania maksimum (ang. *max pooling*) albo do obliczania średniej arytmetycznej (ang. *average pooling*). Każda mapa analizowana jest oddzielnie, więc zawsze pozostajemy w dwóch wymiarach.

Kontynuacja przykładu: [jak działa pooling](https://bfirst.tech/wp-content/uploads/2019/06/sieci_konwolucyjne_2.png)

Dodamy zatem do naszej sieci neuronowej *max pooling*, realizowany za pomocą klasy `nn.MaxPool2d`, z jądrem rozmiaru $3 x 3$ i uzupełnieniem o 1 piksel z każdej strony.

In [ ]:
layers.append(nn.MaxPool2d(3, padding=1))

**Zadanie 1a. Mamy 5 map, a jakiego rozmiaru będą one na tym etapie przetwarzania, jeżeli wejście miało mapę rozmiaru $28 x 28$, a jądro ma rozmiar 3?**

tu odpowiedź

### Spłaszczenie i klasyfikacja

Na tym etapie każdy obiekt przetwarzany przez sieć jest trójwymiarowy, czyli składa się z pewnej liczby dwuwymiarowych map. Do klasyfikacji wykorzystamy sumującą warstwę liniową `nn.Linear`. Nie jest to dla niej odpowiedni format wejścia. Zatem najpierw za pomocą klasy `nn.Flatten` ułożymy piksele jeden za drugim w wektor.

**Zadanie 1b. Klasyfikujemy do 10 klas, więc mamy 10 neuronów wyjściowych, a ilu potrzeba wejść? Uzupełnij kod (w miejscu ...) na podstawie wyników zad. 1.**

In [ ]:
layers.append(torch.nn.Flatten())
layers.append(nn.Linear(..., 10))

### Konfiguracja sieci

**Zadanie 2. Połącz gotowe warstwy w jeden sekwencyjny moduł, jako funkcję straty zadeklaruj entropię krzyżową/skrośną, a do optymalizacji parametrów modelu użyj optymalizatora Adam.**

In [ ]:
model = ...

cost = ...
opt = ...

In [ ]:
model

### Uczenie z wykorzystaniem wczesnego zatrzymania

Wczesne zatrzymanie (ang. *early stopping*) składa się z następujących kroków:
- co określoną liczbę epok uczenia obliczamy miarę oceny (np. trafność klasyfikacji) na zbiorze walidującym
- jeżeli nastąpiła poprawa w stosunku do poprzedniego razu, zapamiętujemy obecne wartości wag sieci neuronowej
- jeżeli przez określoną liczbę epok nie następuje poprawa, przerywamy uczenie i przywracamy wagi ostatniego najlepszego modelu

**Zadanie 3. Poniższy kawałek kodu oblicza trafność na zbiorze walidującym co epokę. Dodaj kod przerywający uczenie, jeżeli przez 5 kolejnych epok nie nastąpiła poprawa. Wyświetlaj wynik i numer porządkowy aktualnie najlepszej epoki, skorzystaj z zadeklarowanych zmiennych, podaj informację o fakcie wczesnego zatrzymania. Pamiętaj że najlepszy model powinien zostać zapisany, sprawdź w dokumentacji biblioteki pytorch jak to zrobić. Na końcu załaduj najlepszy model.**

In [ ]:
train_loss = []
validation_acc = []
best_model = None
best_acc = None
best_epoch = None
max_epoch = 10000
no_improvement = 5
batch_size = 512

for n_epoch in range(max_epoch):
    model.train()
    loader = data.DataLoader(fmnist_train, batch_size=batch_size, shuffle=True, num_workers=1)
    epoch_loss = []
    for X_batch, y_batch in loader:
        opt.zero_grad()
        logits = model(X_batch)
        loss = cost(logits, y_batch)
        loss.backward()
        opt.step()        
        epoch_loss.append(loss.detach())
    train_loss.append(torch.tensor(epoch_loss).mean())
    model.eval()
    loader = data.DataLoader(fmnist_validation, batch_size=len(fmnist_validation), shuffle=False)
    X, y = next(iter(loader))
    logits = model(X)
    acc = compute_acc(logits, y).detach()
    validation_acc.append(acc)
    
    #tutaj napisz kod wczesnego zatrzymania
    
    
model.load_state_dict(best_model)

Przyjrzyj się poniższym wykresom

In [ ]:
plt.title('Trafność na zbiorze walidacyjnym. Kropka oznacza najlepszą trafność.')
plt.plot(validation_acc, label='Trafność na zbiorze walidacyjnym')
plt.plot(best_epoch, best_acc, 'bo', label='Najlepsza trafność')
plt.show()
plt.title('Błąd na zbiorze treningowym')
plt.plot(train_loss)
plt.show()
k = max(3*no_improvement, 0)
plt.title('Ostatnie {} epok'.format(k))
plt.plot(validation_acc[-k:])
plt.plot(best_epoch-(len(validation_acc)-k), best_acc, 'bo')
plt.show()

**Zadanie 4. Dlaczego wczesne zatrzymanie jest realizowane na zbiorze walidującym, a nie na zbiorze uczącym albo na zbiorze testowym?**

tu miejsce na odpowiedź

**Zadanie 5. Przyjrzyj się jeszcze raz kodowi odpowiedzialnemu za epokę uczenia. Wykorzystaj linijki odpowiedzialne tam za obliczanie trafności i oblicz trafność klasyfikacji: na zbiorze walidującym oraz na zbiorze testowym. Czy uzyskane wartości się różnią? Co może być powodem różnic?**

In [ ]:
model.eval()

#tutaj trafność na zbiorze walidacyjnym
print("Dokladnosc na zbiorze walidacyjnym", acc_val)

#tutaj trafność na zbiorze testowym
print("Dokladnosc na zbiorze testowym", acc_test)

**tu napisz komentarz do odpowiedzi**